# Start

In [1]:
import spacy

In [5]:
!uv run spacy download ru_core_news_lg

     ---------------------------------------- 0.0/513.4 MB ? eta -:--:--
     ---------------------------------------- 0.3/513.4 MB ? eta -:--:--
     ---------------------------------------- 1.3/513.4 MB 4.2 MB/s eta 0:02:03
     ---------------------------------------- 2.4/513.4 MB 4.6 MB/s eta 0:01:51
     ---------------------------------------- 3.9/513.4 MB 5.5 MB/s eta 0:01:34
     ---------------------------------------- 5.8/513.4 MB 6.2 MB/s eta 0:01:23
      --------------------------------------- 7.9/513.4 MB 7.1 MB/s eta 0:01:12
      -------------------------------------- 10.2/513.4 MB 7.8 MB/s eta 0:01:05
     - ------------------------------------- 13.4/513.4 MB 8.8 MB/s eta 0:00:57
     - ------------------------------------- 16.8/513.4 MB 9.8 MB/s eta 0:00:51
     - ------------------------------------ 21.2/513.4 MB 11.1 MB/s eta 0:00:45
     - ------------------------------------ 26.2/513.4 MB 12.3 MB/s eta 0:00:40
     -- ----------------------------------- 32.5/513.4

In [6]:
nlp = spacy.load("ru_core_news_lg")

In [7]:
spacy.prefer_gpu() # Используется ли ускорение на gpu

True

## test

In [8]:
text = ("""Используйте трансформерные модели, если нужна максимальная нагрузка на GPU
Стандартные маленькие модели (_sm) часто не используют GPU по полной, для максимума берите _trf версии 
(например, ru_core_news_trf), которые построены на PyTorch и используют CUDA для ускорения.
""")

In [9]:
doc = nlp(text)

In [10]:
# Analyze syntax
print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

Verbs: ['использовать', 'использовать', 'брать', 'построить', 'использовать']


# Data

## Load

In [11]:
import pandas as pd

In [12]:
path_to_train = r"data\train.csv"
df = pd.read_csv(path_to_train, sep=';')

## Convert

In [16]:
import ast
from spacy.tokens import DocBin
import math
import numpy as np

In [17]:
def convert_and_save(data_array, output_path):
    # Создаем объект DocBin для сериализации документов spaCy
    db = DocBin()
    
    # Проходим по всем строкам данных (каждая строка - текст и строка с аннотациями)
    for (text, annotations_str) in data_array:
        # Преобразуем строковое представление аннотаций в Python-объект (список кортежей)
        annotations = ast.literal_eval(annotations_str)

        end_text = len(text)  # Длина текста для корректировки границ сущностей

        # Создаем объект Doc из исходного текста (токенизация без анализа)
        doc = nlp.make_doc(text)

        ents = []
        for start, end, label in annotations:
            # Корректируем границы, чтобы не выходить за пределы текста
            end = end_text if end > end_text else end
            start = 0 if start < 0 else start
            
            # Создаем span - сегмент с меткой, с режимом расширения, чтобы избежать пропусков из-за ошибок разметки
            span = doc.char_span(start, end, label=label, alignment_mode="expand")
            ents.append(span)

        # Присваиваем полученные сущности документу
        doc.ents = ents
        
        # Добавляем документ в объект DocBin (формат для эффективного хранения и обмена)
        db.add(doc)

    # Сохраняем сериализованный набор документов в файл на диск
    db.to_disk(output_path)

In [18]:
# Путь для сохранения обучающего датасета в формате spacy
output_train_path = "data/train.spacy"
# Путь для сохранения валидационного (dev) датасета
output_dev_path = "data/dev.spacy"

# Деление данных DataFrame на тренировочную и дев выборки в соотношении 0.7 к 0.3
data = df.to_numpy()

# Перемешиваем
np.random.seed(42)
shuffled_indices = np.random.permutation(len(data))
data = data[shuffled_indices]

split_idx = math.floor(len(data) * 0.7)  # Индекс, где делим массив

# Разделяем данные
train_data = data[:split_idx]  # 70% для тренировки
dev_data = data[split_idx:]    # 30% для валидации

# Конвертируем и сохраняем тренировочный датасет
convert_and_save(train_data, output_train_path)
# Конвертируем и сохраняем валидационный датасет
convert_and_save(dev_data, output_dev_path)

## Пример считывания из spacy файлы

In [19]:
from spacy.tokens import DocBin

In [20]:
# Загружаем данные из файла
with open(r"data\train.spacy", "rb") as f:
    doc_bin_bytes = f.read()

In [21]:
db = DocBin().from_bytes(doc_bin_bytes)

In [22]:
docs = list(db.get_docs(nlp.vocab))

In [24]:
for doc in docs:
    if doc.text == "abon":
        print(doc.text)
        # Можно получить сущности, токены и др.
        for ent in doc.ents:
            print(ent.text, ent.label_)

abon
abon O


# Train

In [ ]:
# from spacy.tokens import DocBin

# # Чтение и загрузка тренировочного файла
# with open(r"data/train.spacy", "rb") as f:
#     train_doc_bin_bytes = f.read()

# train_db = DocBin().from_bytes(train_doc_bin_bytes)

# # Чтение и загрузка валидирующего файла
# with open(r"data/dev.spacy", "rb") as f:
#     dev_doc_bin_bytes = f.read()

# dev_db = DocBin().from_bytes(dev_doc_bin_bytes)


# Тест обученной модели

In [24]:
import spacy

In [ ]:
nlp = spacy.load("models_gpu/model-last")

In [35]:
doc = nlp(r"абрикосы 500г global village")

In [33]:
doc = nlp("фруктовое пбре без сахара")

In [36]:
for ent in doc.ents:
    print(ent.text, ent.label_)

абрикосы B-TYPE
500 B-BRAND
г I-BRAND
global B-BRAND
village I-BRAND
